In [ ]:
# !pip install langchain_huggingface
# !pip install langchain-community

# Vector Stores

In [16]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [11]:
# Import necessary libraries
from langchain.vectorstores import Chroma

# embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"

from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [17]:
# # Create the Chroma vector store with Hugging Face embeddings
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=hf
)

In [ ]:
# !pip install langchain_ollama

In [18]:
vectorstore.similarity_search("dog")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [19]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.0258983373641968),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
  1.3321926593780518),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
  1.3321928977966309),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6779040098190308)]

In [20]:
embedding = hf.embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

# Retrievers

In [23]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]]

In [24]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]]

In [ ]:
# !pip install langchain_groq

In [26]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

··········


In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [28]:
response = rag_chain.invoke("tell me about cats")

print(response.content)

According to the context, cats are independent pets that often enjoy their own space.


# Building an Agent

#### Tools
Tools are utilities that can be called by a model, and whose outputs are designed to be fed back to a model.
#### Toolkits
LangChain has a concept of toolkits. This a very thin abstraction that groups tools together that are designed to be used together for specific tasks.

#### Agents
By themselves, language models can't take actions - they just output text. Agents are systems that take a high-level task and use an LLM as a reasoning engine to decide what actions to take and execute those actions.

LangGraph is an extension of LangChain specifically aimed at creating highly controllable and customizable agents.

#### A Quick Overview of End to End Agent

In [3]:
import os
import getpass

os.environ["TAVILY_API_KEY"] = getpass.getpass()
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["GOOGLE_API_KEY"] = getpass.getpass()
# os.environ["SERPAPI_API_KEY"] = getpass.getpass()
os.environ["GROQ_API_KEY"] = getpass.getpass()


··········
··········


In [ ]:
# !pip install tavily-python langgraph-checkpoint-sqlite

In [ ]:
# !pip install langchain_anthropic langgraph langchain_community

In [7]:
# Import relevant functionality
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatGroq(model_name="llama3-8b-8192")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yr2c', 'function': {'arguments': '{"query":"San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 948, 'total_tokens': 1019, 'completion_time': 0.059166667, 'prompt_time': 0.111729006, 'queue_time': 0.0071797429999999884, 'total_time': 0.170895673}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6107c5cb-eda0-4252-b084-6f5e5193434d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco'}, 'id': 'call_yr2c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 948, 'output_tokens': 71, 'total_tokens': 1019})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.lonelyplanet.com/articles/top-things-to-do-in-san-francisco", "content": "The 16 best things to do 

## Step By Step Guide

#### Langsmith
Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

In [ ]:
# !pip install langchain_community langchain_core langchain_groq

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1730281744, 'localtime': '2024-10-30 02:49'}, 'current': {'last_updated_epoch': 1730281500, 'last_updated': '2024-10-30 02:45', 'temp_c': 10.0, 'temp_f': 50.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 272, 'wind_dir': 'W', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 25, 'feelslike_c': 8.3, 'feelslike_f': 47.0, 'windchill_c': 9.0, 'windchill_f': 48.2, 'heatindex_c': 10.0, 'heatindex_f': 50.0, 'dewpoint_c': 9.2, 'dewpoint_f': 48.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 11.3, 'gust_kph': 18.1}}"}, {'url': 'https://www.weathertab.com/en/c/e/10

In [ ]:
# !pip install google-search-results serpapi

In [57]:
# from langchain.utilities import GoogleSearchAPIWrapper

# # Make sure to set your GOOGLE_API_KEY and GOOGLE_CSE_ID environment variables.
# google_search = GoogleSearchAPIWrapper()

from langchain.utilities import SerpAPIWrapper

# Make sure to set your SERPAPI_API_KEY environment variable.
bing_search = SerpAPIWrapper(search_engine="bing")

search_results = bing_search.run("what is the weather in Lahore, Pakistan")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
# tools = [bing_search]

{'type': 'weather_result', 'temperature': '88', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '48%', 'wind': '0 mph', 'location': 'Lahore, Pakistan', 'date': 'Wednesday 12:00 PM', 'weather': 'Smoke'}


In [48]:
search_results = search.invoke("search the instagram account with username 'msalmanai62'")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.instagram.com/msalmanai62/', 'content': '48 Followers, 25 Following, 8 Posts - Muhammad Salman (@msalmanai62) on Instagram: "Ai Developer" msalmanai62. Follow. Message. 8 posts. 48 followers. 25 following. Muhammad Salman msalmanai62. Product/service. Ai Developer. 2024. Don\'tGetSerious ... Log in to see photos and videos from friends and discover other accounts you\'ll love.'}, {'url': 'https://toolzu.com/search-instagram-profiles/', 'content': 'Free Search Instagram Profiles Search Instagram Profiles Start Instagram account search for free 1. What is an Instagram search user? 7. Can I do an Instagram reverse image search with this engine? 1. What is an Instagram search user? Instagram account search is a specially developed Instagram search engine that enables convenient and fast search of Instagram profiles by a number of input data (users full names and usernames, number of followers and followings, accounts categories, email addresses, phone numbers, and bio

## Using LLM (Groq here)

In [9]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [10]:
model_with_tools = model.bind_tools(tools)

In [11]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hi!'}, 'id': 'call_zhtz', 'type': 'tool_call'}]


In [12]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Lahore, Pakistan?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Lahore, Pakistan'}, 'id': 'call_v1eh', 'type': 'tool_call'}]


In [ ]:
# !pip install langchain-groq --upgrade

## Creating agent

In [ ]:
# from langchain import hub
# prompt = hub.pull("ih/ih-react-agent-executor")
# prompt.pretty_print()

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools=tools,
                                    state_modifier="Answer the following question."
                                    )

response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what is the weather in SF?")]}
)
response["messages"]

[HumanMessage(content='what is the weather in SF?', additional_kwargs={}, response_metadata={}, id='b255749a-104f-4bb6-85aa-cb2565a975a5'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qs97', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1929, 'total_tokens': 2002, 'completion_time': 0.060833333, 'prompt_time': 0.088701049, 'queue_time': -0.17881243800000002, 'total_time': 0.149534382}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7802b326-ff3a-4567-b5f2-ea691e0d5179-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_qs97', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1929, 'output_tokens': 73, 'total_tokens': 2002}),
 ToolMessage(content='[{"url": "h

#### Streaming the output

In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="what is the weather in Lahore, Pakistan?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sxsk', 'function': {'arguments': '{"query":"weather in Lahore, Pakistan"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 956, 'total_tokens': 1037, 'completion_time': 0.0675, 'prompt_time': 0.143460976, 'queue_time': 0.12792292200000002, 'total_time': 0.210960976}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-930d4495-820a-4586-8591-14347d3de02a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Lahore, Pakistan'}, 'id': 'call_sxsk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 956, 'output_tokens': 81, 'total_tokens': 1037})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Lahore\', \'region\'

#### Streaming tokens

In [18]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="what is the weather in Lahore, Pakistan?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Tool use failed: no tool can be called with name open_weather_map|

In [19]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hey Bob! How's it going?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 1905, 'total_tokens': 1914, 'completion_time': 0.008306031, 'prompt_time': 0.098999949, 'queue_time': -0.18303395900000002, 'total_time': 0.10730598}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-e3244e34-5c79-4751-b405-d6340708da3d-0', usage_metadata={'input_tokens': 1905, 'output_tokens': 9, 'total_tokens': 1914})]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 966, 'total_tokens': 972, 'completion_time': 0.005, 'prompt_time': 0.044415539, 'queue_time': 0.0018194810000000047, 'total_time': 0.049415539}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7056dbfb-2302-4aca-abba-dcc1630abda5-0', usage_metadata={'input_tokens': 966, 'output_tokens': 6, 'total_tokens': 972})]}}
----


starting new chat by just replacing new thread_id

In [23]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3c4q', 'function': {'arguments': '{"query":"what is my name"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 944, 'total_tokens': 1017, 'completion_time': 0.060833333, 'prompt_time': 0.109469267, 'queue_time': -0.183685931, 'total_time': 0.1703026}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e73c9b55-ac9b-470a-94bd-c2bb9513ca69-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'what is my name'}, 'id': 'call_3c4q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 944, 'output_tokens': 73, 'total_tokens': 1017})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://quizondo.com/what-is-my-name-quiz/", "content": "Miscellaneous\\nIf the question “What Is My Name?” has been p